- https://www.diva-portal.org/smash/get/diva2:21391/FULLTEXT01.pdf
- enthält refomrulierung as set partioon problem

## Introduction

Given a set of agents and a set of tasks and for each pair (agent, task) a costs or profit value. Moreover each agent has a budget, whcih means that the sum of costs assigned to an agent can not exceed his budget.

The goal is to find an assigment of agents to tasks such that the overall profit is maximized. (More precisely each task is assigned to only one agent.)

Its known that the generalized assignment problem is NP-hard.

### Special cases

1. If all agent budgets and tasks costs are equal to one, then the problem is the [assignment problem](https://en.wikipedia.org/wiki/Assignment_problem)
1. If consts and profits of all tasks are equal for each agent, then the problem reduces to the [multiple knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem).
1. If there is only a single agent, then the problem reduces to the [knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem)

Further introductory reading: https://en.wikipedia.org/wiki/Generalized_assignment_problem

## Abstract mathematical formulation (as integer program)

$$
\begin{array}{llr}
\min & \sum_{i,j} c_{ij} x_{ij} & \\
s.t. & \sum_j w_{ij} x_{ij} \leq t_i & \forall i\\
     & \sum_i x_{ij} \leq 1 & \forall j
\end{array}
$$

where
- $I$ index set of agents
- $J$ index set of tasks
- $c_{ij}$ cost/profit of assigning agent $i$ to task $j$
- $w_{ij}$ weight
- $t_i$ capacity of agent $i$

### decision variables
